# AI - Vacuum agent, empty program

Created (R. Basili, Nov. 2020)  
Modified (C.D. Hromei, Oct 2024)

## Exercises

1. Fill the program of the `Agent` in order to reproduce the behaviour of random moves.
2. Fill the program of the `Agent` in order to search for the closest dirty room, go there and `Suck`.  
    **Hint**: you should modify the `Perception` method of the Environment as well to make it Completely Observable.
3. Modify the `Environment` locations into a 2x2 matrix instead of a row.
4. Modify the `Execute action` method of the Environment in order to implement a probabilistic failure of the action chosen by the Agent. In that case, the action should fail and nothing happens!

# Soluzione Es3

In questo caso a differenza degli altri due Esercizi ho dovuto inanzitutto aggiungere una nuova loc_D e cambiare le coordinate delle vecchie locazioni, 
Successivamente ho dovuto aggiornare la GUI per far in modo che ci fossero 4 blocchi disposti a 2 a 2 e non più 3 blocchi in fila,
Ho aggiunto i comandi `Top` e `Bottom` per far sì che il robottino si spostasse sopra o sotto,
Ho dovuto aggiornare la lettura dell'ambiente per far in modo che leggesse anche il quarto blocco aggiunto
e infine ho dovuto aggiornare il comportamento del robottino:

``` python
if status[value] == 'Dirty':
            return 'Suck'
        
        if location == loc_A:
            if status[1] == 'Dirty':
                return 'Right'
            elif status[2] == 'Dirty':
                return 'Bottom'
            elif status[3] == 'Dirty':
                return 'Right'
            
        if location == loc_B:
            if status[0] == 'Dirty':
                return 'Left'
            elif status[3] == 'Dirty':
                return 'Bottom'
            elif status[2] == 'Dirty':
                return 'Left'
            
        if location == loc_C:
            if status[3] == 'Dirty':
                return 'Right'
            elif status[0] == 'Dirty':
                return 'Top'
            elif status[1] == 'Dirty':
                return 'Right'
            
        if location == loc_D:
            if status[2] == 'Dirty':
                return 'Left'
            elif status[1] == 'Dirty':
                return 'Top'
            elif status[0] == 'Dirty':
                return 'Left' 
```
Prima di tutto controlla se la cella è sporca in quel caso la pulisce, 
Altrimenti poiché l'ambiente è completamente osservabile in base alla posizione dove sta controlla le altre celle e se ne trova una sporca si muove di conseguenza per andare a pulirla
Prendiamo per esempio la loc_A:

```python
if location == loc_A:
            if status[1] == 'Dirty':
                return 'Right'
            elif status[2] == 'Dirty':
                return 'Bottom'
            elif status[3] == 'Dirty':
                return 'Right'
```
Se siamo nella loc_A (alto sx) e status[1] che sarebbe la loc_B (alto dx) è sportca ci spostiamo a destra, altrimenti se status[2] che sarebbe loc_C (basso sx) è sporco ci spostiamo sotto tramite il nuovo comando aggiunto `Bottom`, infine se status[3] che sarebbe loc_D (basso dx) è sporco poichè non possiamo arrivarci direttamente facciamo prima uno spostamento a destra


In [1]:
from tkinter import *
import random
from agents import *

loc_A, loc_B, loc_C, loc_D = (0, 0), (0, 1), (1,0), (1,1)  # Modifica della matrice 2x2

#MyImage = None va tolto per aggiungere immagine

class Gui(Environment):

    """This GUI environment has three locations, A, B and C. Each can be Dirty
    or Clean. The agent perceives its location and the location's
    status."""

    def __init__(self, root, height=600, width=600):
        super().__init__()
        self.status = {loc_A: 'Dirty',
                       loc_B: 'Dirty', 
                       loc_C: 'CLean',
                       loc_D: 'Clean'}
        self.root = root
        self.height = height
        self.width = width
        self.canvas = None
        self.buttons = []
        self.create_canvas()
        self.create_buttons()

    def thing_classes(self):
        """The list of things which can be used in the environment."""
        return [Wall, Dirt, ReflexVacuumAgent, RandomVacuumAgent,
                TableDrivenVacuumAgent, ModelBasedVacuumAgent]

    def percept(self, agent):
        """Returns the agent's location, and the location status (Dirty/Clean)."""
        print('Perceived Location=', agent.location,' - Perceived StatusAtLoc=',  self.status[agent.location])
        return (agent.location, self.status) #(agent.location, self.status[agent.location]) #In questo modo ritorno tutto l'array e lo rendo completamente osservabile

    def execute_action(self, agent, action):
        """Change the location status (Dirty/Clean); track performance.
        Score 10 for each dirt cleaned; -1 for each move."""
        print('Executing action: ', action)
        if action == 'Right':
            if agent.location == loc_A:
                agent.location=loc_B
            elif agent.location == loc_C:
                agent.location=loc_D
            agent.performance -= 1
        elif action == 'Left':
            if agent.location == loc_B:
                agent.location = loc_A
            elif agent.location == loc_D:
                agent.location=loc_C
            agent.performance -= 1
        elif action == 'Top':
            if agent.location == loc_C:
                agent.location = loc_A
            elif agent.location == loc_D:
                agent.location=loc_B
            agent.performance -= 1
        elif action == 'Bottom':
            if agent.location == loc_A:
                agent.location = loc_C
            elif agent.location == loc_B:
                agent.location=loc_D
            agent.performance -= 1
            
        elif action == 'Suck':
            if self.status[agent.location] == 'Dirty':
                if agent.location == loc_A:
                    self.buttons[0].config(bg='white', activebackground='light grey')
                elif agent.location == loc_B:
                    self.buttons[1].config(bg='white', activebackground='light grey')
                elif agent.location == loc_C:
                    self.buttons[2].config(bg='white', activebackground='light grey')
                else:
                    self.buttons[3].config(bg='white', activebackground='light grey')
                
                agent.performance += 10
            self.status[agent.location] = 'Clean'

    def default_location(self, thing):
        """Agents start in either location at random."""
        return random.choice([loc_A, loc_B, loc_C, loc_D])

    ###################################################################################################
    #                        INTERFACE METHODS NOT USEFULL FOR THE AGENT/ENVIRONMENT
    ###################################################################################################
    def create_canvas(self):
        """Creates Canvas element in the GUI."""
        self.canvas = Canvas(
            self.root,
            width=self.width,
            height=self.height,
            background='powder blue')
        self.canvas.pack(side='bottom')

    def create_buttons(self):
        """Creates the buttons required in the GUI."""
        button_lefttop = Button(self.root, height=4, width=12, padx=2, pady=2, bg='white')
        button_lefttop.config(command=lambda btn=button_lefttop: self.dirt_switch(btn))
        self.buttons.append(button_lefttop)
        button_lefttop_window = self.canvas.create_window(190, 190, anchor=N, window=button_lefttop)
        
        button_righttop = Button(self.root, height=4, width=12, padx=2, pady=2, bg='white')
        button_righttop.config(command=lambda btn=button_righttop: self.dirt_switch(btn))
        self.buttons.append(button_righttop)
        button_righttop_window = self.canvas.create_window(310, 190, anchor=N, window=button_righttop)

        button_leftbottom = Button(self.root, height=4, width=12, padx=2, pady=2, bg='white')
        button_leftbottom.config(command=lambda btn=button_leftbottom: self.dirt_switch(btn))
        self.buttons.append(button_leftbottom)
        button_leftbottom_window = self.canvas.create_window(190, 310, anchor=N, window=button_leftbottom)

        button_rightbottom = Button(self.root, height=4, width=12, padx=2, pady=2, bg='white')
        button_rightbottom.config(command=lambda btn=button_rightbottom: self.dirt_switch(btn))
        self.buttons.append(button_rightbottom)
        button_rightbottom_window = self.canvas.create_window(310, 310, anchor=N, window=button_rightbottom)

    def dirt_switch(self, button):
        """Gives user the option to put dirt in any tile."""
        bg_color = button['bg']
        if bg_color == 'saddle brown':
            button.config(bg='white', activebackground='light grey')
        elif bg_color == 'white':
            button.config(bg='saddle brown', activebackground='light goldenrod')

    def read_env(self):
        """Reads the current state of the GUI."""
        for i, btn in enumerate(self.buttons):
            if i == 0:
                if btn['bg'] == 'white':
                    self.status[loc_A] = 'Clean'
                else:
                    self.status[loc_A] = 'Dirty'
            elif i == 1:
                if btn['bg'] == 'white':
                    self.status[loc_B] = 'Clean'
                else:
                    self.status[loc_B] = 'Dirty'
            elif i == 2:
                if btn['bg'] == 'white':
                    self.status[loc_C] = 'Clean'
                else:
                    self.status[loc_C] = 'Dirty'
            else:
                if btn['bg'] == 'white':
                    self.status[loc_D] = 'Clean'
                else:
                    self.status[loc_D] = 'Dirty'

    def update_env(self, agent):
        """Updates the GUI according to the agent's action."""
        self.read_env()
        # print(self.status)
        before_step = agent.location
        self.step()   #execute_actions
        print('Current Environment Status: ', self.status)
        print('Current Agent Location: ',agent.location)
        move_agent(self, agent, before_step)

###################################################################################################
#                                           THE AGENT
###################################################################################################

def ReflexVacuumAgent():
    """A reflex agent for the two-state vacuum environment. [Figure 2.8]
    >>> agent = ReflexVacuumAgent()
    >>> environment = TrivialVacuumEnvironment()
    >>> environment.add_thing(agent)
    >>> environment.run()
    >>> environment.status == {(1,0):'Clean' , (0,0) : 'Clean'}
    True
    """
    
    def program(percept):
        
        location, status = percept
        print('Perceived Status: ', percept)
        status = list(status.values())
        value = location[0]*2+location[1]
        
        if status[value] == 'Dirty':
            return 'Suck'
        
        if location == loc_A:
            if status[1] == 'Dirty':
                return 'Right'
            elif status[2] == 'Dirty':
                return 'Bottom'
            elif status[3] == 'Dirty':
                return 'Right'
            
        if location == loc_B:
            if status[0] == 'Dirty':
                return 'Left'
            elif status[3] == 'Dirty':
                return 'Bottom'
            elif status[2] == 'Dirty':
                return 'Left'
            
        if location == loc_C:
            if status[3] == 'Dirty':
                return 'Right'
            elif status[0] == 'Dirty':
                return 'Top'
            elif status[1] == 'Dirty':
                return 'Right'
            
        if location == loc_D:
            if status[2] == 'Dirty':
                return 'Left'
            elif status[1] == 'Dirty':
                return 'Top'
            elif status[0] == 'Dirty':
                return 'Left' 
        
    return Agent(program)

###################################################################################################
#                            INTERFACE METHODS TO CREATE AND MOVE THE AGENT
###################################################################################################

def create_agent(env, agent, MyImage):
    """Creates the agent in the GUI and is kept independent of the environment."""
    env.add_thing(agent)

    if agent.location == (0, 0):
        env.text = env.canvas.create_text(180, 150, font="Helvetica 10 bold italic", text="Agent")
        env.agent_logo = env.canvas.create_image(180, 150, image=MyImage)
    elif agent.location == (0, 1):
        env.text = env.canvas.create_text(300, 150, font="Helvetica 10 bold italic", text="Agent")
        env.agent_logo = env.canvas.create_image(300, 150, image=MyImage)
    elif agent.location == (1, 0):
        env.text = env.canvas.create_text(180, 430, font="Helvetica 10 bold italic", text="Agent")
        env.agent_logo = env.canvas.create_image(180, 430, image=MyImage)
    elif agent.location == (1, 1):
        env.text = env.canvas.create_text(300, 430, font="Helvetica 10 bold italic", text="Agent")
        env.agent_logo = env.canvas.create_image(300, 430, image=MyImage)
    
def move_agent(env, agent, before_step):
    """Moves the agent in the GUI when 'next' button is pressed."""
    if agent.location == before_step:
        pass
    else:
        if (agent.location == (1, 0) and  before_step == (0,0)) or (agent.location == (1, 1) and  before_step == (0,1)): 
            env.canvas.move(env.text, 0, 280)
            env.canvas.move(env.agent_logo, 0, 280)
        elif (agent.location == (0, 0) and before_step == (1,0)) or (agent.location == (0, 1) and before_step == (1,1)):
            env.canvas.move(env.text, 0, -280)
            env.canvas.move(env.agent_logo, 0, -280)
        elif (agent.location == (0, 0) and before_step == (0,1)) or (agent.location == (1, 0) and before_step == (1,1)):
            env.canvas.move(env.text, -120, 0)
            env.canvas.move(env.agent_logo, -120, 0)
        elif (agent.location == (0, 1) and before_step == (0,0)) or (agent.location == (1, 1) and before_step == (1,0)):
            env.canvas.move(env.text, 120, 0)
            env.canvas.move(env.agent_logo, 120, 0)

# TODO: Add more agents to the environment.
# TODO: Expand the environment to XYEnvironment.

###################################################################################################
#                                           MAIN LOOP
###################################################################################################
def main():
    """The main function of the program."""
    root = Tk()
    root.title("Vacuum Environment")
    root.geometry("600x600")
    root.resizable(0, 0)
    frame = Frame(root, bg='black')
    
    MyImage = PhotoImage(file="AgentVacuum.png")
    
    next_button = Button(frame, text='Next', height=2, width=6, padx=2, pady=2)
    next_button.pack(side='left')
    frame.pack(side='bottom')
    
    env = Gui(root)
    agent = ReflexVacuumAgent()
    create_agent(env, agent, MyImage)
    
    next_button.config(command=lambda: env.update_env(agent))
    
    root.mainloop()


if __name__ == "__main__":
    main()


Perceived Location= (0, 0)  - Perceived StatusAtLoc= Dirty
Perceived Status:  ((0, 0), {(0, 0): 'Dirty', (0, 1): 'Dirty', (1, 0): 'Clean', (1, 1): 'Dirty'})
Executing action:  Suck
Current Environment Status:  {(0, 0): 'Clean', (0, 1): 'Dirty', (1, 0): 'Clean', (1, 1): 'Dirty'}
Current Agent Location:  (0, 0)
Perceived Location= (0, 0)  - Perceived StatusAtLoc= Clean
Perceived Status:  ((0, 0), {(0, 0): 'Clean', (0, 1): 'Dirty', (1, 0): 'Clean', (1, 1): 'Dirty'})
Executing action:  Right
Current Environment Status:  {(0, 0): 'Clean', (0, 1): 'Dirty', (1, 0): 'Clean', (1, 1): 'Dirty'}
Current Agent Location:  (0, 1)
Perceived Location= (0, 1)  - Perceived StatusAtLoc= Dirty
Perceived Status:  ((0, 1), {(0, 0): 'Clean', (0, 1): 'Dirty', (1, 0): 'Clean', (1, 1): 'Dirty'})
Executing action:  Suck
Current Environment Status:  {(0, 0): 'Clean', (0, 1): 'Clean', (1, 0): 'Clean', (1, 1): 'Dirty'}
Current Agent Location:  (0, 1)
Perceived Location= (0, 1)  - Perceived StatusAtLoc= Clean
Perceiv